In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "trl<0.9.0" xformers

In [2]:
import unsloth
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Phi-3-mini-4k-instruct",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.9.4: Fast Mistral patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank - higher for better adaptation
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0.05, # Small dropout for regularization
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.9.4 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [5]:
from datasets import load_dataset
dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split="train[:1000]")

README.md:   0%|          | 0.00/905 [00:00<?, ?B/s]

data/train-00000-of-00001-8b6e212f3e1ece(…):   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

In [6]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'prompt'],
    num_rows: 1000
})

In [21]:
dataset['instruction'][0]

'Create a function to calculate the sum of a sequence of integers.'

In [22]:
dataset['input'][0]

'[1, 2, 3, 4, 5]'

In [24]:
print(dataset['output'][0])

# Python code
def sum_sequence(sequence):
  sum = 0
  for num in sequence:
    sum += num
  return sum


In [26]:
print(dataset['prompt'][0])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a function to calculate the sum of a sequence of integers.

### Input:
[1, 2, 3, 4, 5]

### Output:
# Python code
def sum_sequence(sequence):
  sum = 0
  for num in sequence:
    sum += num
  return sum


In [7]:
def format_chat_template(examples):
    """Format examples using Phi-3's specific chat template"""
    texts = []

    for i in range(len(examples["instruction"])):
        instruction = examples["instruction"][i]
        output = examples["output"][i]

        # Phi-3 uses specific chat template format
        text = f"""<|user|>
{instruction}<|end|>
<|assistant|>
{output}<|end|>"""

        texts.append(text)

    return {"text": texts}


formatted_dataset = dataset.map(
    format_chat_template,
    batched=True,
    remove_columns=dataset.column_names,
    desc="Formatting dataset for Phi-3"
)

Formatting dataset for Phi-3:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
formatted_dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [27]:
print(formatted_dataset[0]['text'])

<|user|>
Create a function to calculate the sum of a sequence of integers.<|end|>
<|assistant|>
# Python code
def sum_sequence(sequence):
  sum = 0
  for num in sequence:
    sum += num
  return sum<|end|>


In [10]:
from transformers import TrainingArguments
from trl import SFTTrainer

In [12]:
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=100,  # Short training for demo
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",  # Memory-efficient optimizer
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="phi3_finetuned",
    save_strategy="no",
)

In [13]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    args=training_args,
    packing=False,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [14]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416 of 3,850,963,968 (0.78% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abeshith (abeshith-dr-m-g-r-educational-and-research-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.060100
2,0.900500
3,1.090400
4,0.884200
5,0.920200
6,0.857900
7,0.691900
8,0.740000
9,0.798200
10,0.603000


TrainOutput(global_step=100, training_loss=0.6485291329026223, metrics={'train_runtime': 502.0991, 'train_samples_per_second': 1.593, 'train_steps_per_second': 0.199, 'total_flos': 4660829670481920.0, 'train_loss': 0.6485291329026223, 'epoch': 0.8})

In [15]:
model.save_pretrained("phi3_python_tutor")
tokenizer.save_pretrained("phi3_python_tutor")

('phi3_python_tutor/tokenizer_config.json',
 'phi3_python_tutor/special_tokens_map.json',
 'phi3_python_tutor/chat_template.jinja',
 'phi3_python_tutor/tokenizer.model',
 'phi3_python_tutor/added_tokens.json',
 'phi3_python_tutor/tokenizer.json')

In [ ]:
FastLanguageModel.for_inference(model)

In [17]:
def test_phi3_model(instruction, max_tokens=256):
    """Test the fine-tuned Phi-3 model with a coding instruction"""

    prompt = f"""<|user|>
{instruction}<|end|>
<|assistant|>
"""

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.convert_tokens_to_ids("<|end|>"),
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_response = response.split("<|assistant|>")[-1].replace("<|end|>", "").strip()

    return assistant_response

In [18]:
test_cases = [
    "Write a Python function to calculate the factorial of a number using recursion.",
    "Create a Python class for a simple calculator with add, subtract, multiply, and divide methods.",
    "Write a Python function to find the largest element in a list without using max() function.",
    "Explain how to handle exceptions in Python with a try-except block example.",
    "Write a Python function to reverse a string without using built-in reverse methods."
]

for i, test_case in enumerate(test_cases, 1):
    print(f"\n🔍 Test {i}:")
    print(f"Q: {test_case}")
    print(f"A: {test_phi3_model(test_case, max_tokens=200)}")
    print("-" * 50)


🔍 Test 1:
Q: Write a Python function to calculate the factorial of a number using recursion.
A: Write a Python function to calculate the factorial of a number using recursion. def factorial(num):
    if num == 0:
        return 1
    else:
        return (num * factorial(num-1))
--------------------------------------------------

🔍 Test 2:
Q: Create a Python class for a simple calculator with add, subtract, multiply, and divide methods.
A: Create a Python class for a simple calculator with add, subtract, multiply, and divide methods. class Calculator:
    def __init__(self):
        self.value = 0

    def add(self, n):
        self.value += n
        return self.value

    def subtract(self, n):
        self.value -= n
        return self.value

    def multiply(self, n):
        self.value *= n
        return self.value

    def divide(self, n):
        self.value /= n
        return self.value
--------------------------------------------------

🔍 Test 3:
Q: Write a Python function 

In [28]:
if torch.cuda.is_available():
    print(f"\n📊 GPU Memory Usage:")
    print(f"Allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
    print(f"Reserved: {torch.cuda.memory_reserved()/1024**3:.2f} GB")


📊 GPU Memory Usage:
Allocated: 2.51 GB
Reserved: 3.41 GB
